In [1]:
import os
%pwd

'd:\\end-to-end-ml-project-with-mlflow\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\end-to-end-ml-project-with-mlflow'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig():
    root_dir:Path
    data_path:Path

In [4]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager():
    def __init__(self,config_filepath=CONFIG_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

In [6]:
from mlProject import logger
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler

In [7]:
class DataTransformation():
    def __init__(self,config:DataTransformationConfig):
        self.config = config

    def train_test_splitting(self):
        
        data = pd.read_csv(self.config.data_path)

        # Treating outliers below...
        def log_transform(col):
            return np.log(col[0])

        data['residual sugar'] = data[['residual sugar']].apply(log_transform, axis=1)
        data['chlorides'] = data[['chlorides']].apply(log_transform, axis=1)
        data['free sulfur dioxide'] = data[['free sulfur dioxide']].apply(log_transform, axis=1)
        data['total sulfur dioxide'] = data[['total sulfur dioxide']].apply(log_transform, axis=1)
        data['sulphates'] = data[['sulphates']].apply(log_transform, axis=1)

        # Handling imbalanced dataset below...
        df_3 = data[data.quality==3]     # MINORITY
        df_4 = data[data.quality==4]     # MINORITY
        df_5 = data[data.quality==5]     # MAJORITY
        df_6 = data[data.quality==6]     # MAJORITY
        df_7 = data[data.quality==7]     # MINORITY
        df_8 = data[data.quality==8]     # MINORITY

        # Oversample MINORITY Class to make balance data :
        df_3_upsampled = resample(df_3, replace=True, n_samples=600, random_state=12) 
        df_4_upsampled = resample(df_4, replace=True, n_samples=600, random_state=12) 
        df_7_upsampled = resample(df_7, replace=True, n_samples=600, random_state=12) 
        df_8_upsampled = resample(df_8, replace=True, n_samples=600, random_state=12) 
        # Decreases the rows of Majority one's to make balance data :
        df_5_downsampled = data[data.quality==5].sample(n=600).reset_index(drop=True)
        df_6_downsampled = data[data.quality==6].sample(n=600).reset_index(drop=True)
        # Combine downsampled majority class with upsampled minority class
        Balanced_df = pd.concat([df_3_upsampled, df_4_upsampled, df_7_upsampled, df_8_upsampled, df_5_downsampled, df_6_downsampled]).reset_index(drop=True) 

        train, test = train_test_split(Balanced_df,test_size=0.2,random_state=42)
        
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index=False,header=True)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index=False,header=True)

        print(train.head())
        print(test.head())

        num_pipeline= Pipeline(
            steps=[
            ("imputer",SimpleImputer(strategy="median")),
            ("scaler",StandardScaler())

            ]
        )

        target_column_name="quality"
        numerical_columns = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
            'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
            'pH', 'sulphates', 'alcohol', 'quality']

        input_feature_train_df=train.drop(columns=[target_column_name],axis=1)
        target_feature_train_df=train[target_column_name]

        input_feature_test_df=test.drop(columns=[target_column_name],axis=1)
        target_feature_test_df=test[target_column_name]

        logger.info(
            f"Applying preprocessing object on training dataframe and testing dataframe."
        )

        input_feature_train_arr=num_pipeline.fit_transform(input_feature_train_df)
        input_feature_test_arr=num_pipeline.transform(input_feature_test_df)

        train_arr = np.c_[input_feature_train_arr, np.array(target_feature_train_df)]
        test_arr = np.c_[input_feature_test_arr, np.array(target_feature_test_df)]


        logger.info("Splitted the data into test & training data")
        logger.info(f"Shape of the training data is : {train.shape}")
        logger.info(f"Shape of the test data is : {test.shape}")

        print(train.shape)
        print(test.shape)

        return(
            train_arr,
            test_arr,
        )

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
    data_transformation.transforming()
except Exception as e:
    raise e

[2023-12-11 00:44:05,169: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-11 00:44:05,169: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-11 00:44:05,169: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-11 00:44:05,181: INFO: common: created directory at: artifacts]
[2023-12-11 00:44:05,181: INFO: common: created directory at: artifacts/data_transformation]


C:\Users\vedan\AppData\Local\Temp\ipykernel_11680\203955527.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.log(col[0])
C:\Users\vedan\AppData\Local\Temp\ipykernel_11680\203955527.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.log(col[0])
C:\Users\vedan\AppData\Local\Temp\ipykernel_11680\203955527.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.log(col[0])
C:\Users\vedan\AppData\Local\Te

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
3281            7.5             0.310         0.41        0.875469  -2.733368   
2383            7.9             0.350         0.46        1.280934  -2.551046   
2009           10.7             0.350         0.53        0.955511  -2.659260   
2114            7.4             0.360         0.30        0.587787  -2.603690   
1128            8.4             0.635         0.36        0.693147  -2.419119   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
3281             3.526361              4.094345  0.99492  3.34  -0.162519   
2383             2.708050              3.610918  0.99730  3.35  -0.150823   
2009             1.609438              2.772589  0.99720  3.15  -0.430783   
2114             2.833213              3.178054  0.99419  3.24  -0.356675   
1128             2.708050              4.007333  0.99745  3.31  -0.562119   

      alcohol  quality  
3281     11.4        6  


AttributeError: 'DataTransformation' object has no attribute 'transforming'

In [ ]:
data